In [ ]:
#TAOP89 Marcus Döberl (mardo435) York F.v Wangenheim (yorfr185)
#Tar emot filer och lägger till .mat, används med mat filer
import numpy as np
import time
import sys
import copy
import scipy.io

e=0.01
print("enter file name: ", end="")
fil = input()  
prob = fil
fil= fil +".mat"
print("Reading data from file '" + fil + "'")

if fil==None:
    print('File '+repr(fil)+' does n ot exist')
    exit()

print('Datafile: '+repr(fil))
mat=scipy.io.loadmat(fil,squeeze_me=True)

m=mat['m'] #antal möjliga locations för en fabrik
n=mat['n'] #antalet kunder 
s=mat['s'] #kapaciteten hos en fabrik på plats i    
d=mat['d'] #efterfrågan hos en kund j
f=mat['f'] #fasta kostnade för en anläggning på plats i
c=mat['c'] #transportkostnaden per enhet till kund j från anläggning i
# print ('m:',m,' n:',n)
# print ('s:',s)
# print ('d:',d)
# print ('f:',f)
# print ('c:',c)

t1=time.time()
x=np.zeros((m,n),dtype=np.int)
y=np.zeros((m),dtype=np.int)

ss=copy.deepcopy(s)
ff=copy.deepcopy(f)
cc=copy.deepcopy(c)
dd=copy.deepcopy(d)

while sum(dd)>0:
    fabrik = np.argmin(ff)
    y[fabrik] = 1
    kund = np.where(cc[fabrik,:] == np.amin(cc[fabrik,:]))[0][0]
    if(ss[fabrik] > dd[kund]): #Om fabriken kan ta hand om hela kundens efterfrågan
            ss[fabrik] -= dd[kund]
            x[fabrik,kund] = dd[kund]
            dd[kund] = 0
            cc[:,kund] = 2147483647
    elif(ss[fabrik] <= dd[kund]): # Om fabriken inte kan det, använd den så mycket som möjligt
        dd[kund] -= ss[fabrik]
        x[fabrik,kund] = ss[fabrik]
        ss[fabrik] = 0 #Fabrikens kapacitet är nu slut
        if(dd[kund] == 0):
            cc[:,kund] = 2147483647
        ff[fabrik] = 2147483647
        cc[fabrik,:] = 2147483647 #Sätter alla transportkostnander från den tomma fabriken till oöndligheten


    
elapsed = time.time() - t1
print ('Tid: '+str('%.4f' % elapsed + "s"))

cost=sum(sum(np.multiply(c,x))) + e*np.dot(f,y)
print ('Problem:',prob,' Totalkostnad: '+str(cost))
differans = np.subtract(106668,cost)
print('Differans', differans)
print ('y:',y)
print ('x:',x)
print ('Antal byggda fabriker:',sum(y),'(av',m,')')